## Import required packages

In [1]:
import os
import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf

from data.abc import ABCPreProcessor, ABCTokenizer
from models.symbolic.rnn import FolkLSTM

## Required files and directories

In [2]:
# Mention the path to the datastore
BASE_DIR = "/home/rithomas/project/AI-Music-Generation-Challenge-2020/"
PROCESSED_ABC_FILENAME = 'processed-abc-files'

#BASE_DIR = "/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/AI-Music-Generation-Challenge-2020"
#ABC_TFRECORD_DIR = os.path.join("/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020", "tfrecords", "abc")
#PROCESSED_ABC_FILENAME = 'processed-abc-files'

## Load preprocessed dataset

In [3]:
ABC_TFRECORD_DIR = os.path.join("/home/rithomas/cache", "ABC", "6_8_Meter")
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
six_eight_meter_dataset = preprocessor.load_tfrecord_dataset()

DOUBLE_JIGS_TFRECORD_DIR = os.path.join("/home/rithomas/cache", "ABC", "Double_Jigs")
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
double_jigs_dataset = preprocessor.load_tfrecord_dataset()

<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>
<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>


# Folk-LSTM

In [4]:
batch_size = 128
training = True
FOLK_LSTM_DIR = os.path.join(BASE_DIR, 'configs', 'lstm_512')

data_dims = preprocessor.get_data_dimensions(ABC_TFRECORD_DIR)
data_dims['batch_size'] = batch_size
dataset = preprocessor.prepare_dataset(six_eight_meter_dataset, batch_size)
print(data_dims)

DEFAULT_LR_CONFIG = {
    'initial_lr': 0.001,
    'final_lr': 0.0001,
    'decay_steps': 20000,
}
model = FolkLSTM(FOLK_LSTM_DIR, data_dims, ABC_TFRECORD_DIR, training, DEFAULT_LR_CONFIG)
print(model.get_configs())


## TRAINING PHASE
print('========== TRAINING PHASE ===========')
TRAIN_CONFIG = {
    'print_outputs_frequency': 100,
    'save_frequency': 1000,
    'validation_freq': 1000,
    'max_steps_for_model': 20000
}
#model.train(dataset, TRAIN_CONFIG)

## FINE TUNING PHASE
print('========== FINE TUNING PHASE ===========')
FINE_TUNE_CONFIG = {
    'print_outputs_frequency': 100,
    'save_frequency': 1000,
    'validation_freq': 1000,
    'max_steps_for_model': 22000
}
dataset = preprocessor.prepare_dataset(double_jigs_dataset, batch_size)
#model.train(dataset, FINE_TUNE_CONFIG)

{'max_timesteps': 255, 'tune_vocab_size': 167, 'batch_size': 128}
/home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_512/lstm.json
{'tune_embedding_size': '16', 'rnn': DictWrapper({'num_layers': '3', 'num_units': '512', 'unit_type': 'lstm'})}
Restored from /home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_512/ckpt/ckpt-22
{'tune_embedding_size': '16', 'rnn': DictWrapper({'num_layers': '3', 'num_units': '512', 'unit_type': 'lstm'})}
========== TRAINING PHASE ===========
========== FINE TUNING PHASE ===========


# Generate 10000 double jigs!

In [5]:
from datetime import datetime
start = datetime.now()

MAX_WARNINGS = 1
i = 0
#n = 100
n = 10000
start_tokens = ['<s>', 'M:6/8']
temperature = 1
model = FolkLSTM(FOLK_LSTM_DIR, data_dims, ABC_TFRECORD_DIR, False)

abc_file = open(os.path.join(ABC_TFRECORD_DIR, "10000_double_jigs.abc"), 'w', buffering=20*(1024**2))
temp_fp = os.path.join(ABC_TFRECORD_DIR, "temp_abc.abc")

while(i <= n):
    print('-----------------------------------------------------')
    tune = model.complete_tune(start_tokens, temperature)
    formatted_tune = 'X:' + str(i) + '\n' + tune[1] + '\n' + tune[2] + '\n' + 'L:1/8' + '\n' + ''.join(tune[3:len(tune)-1])
    with open(temp_fp, 'w') as temp_file:
        temp_file.write(formatted_tune)

    valid_tune = True
    error_logs = !abc2midi '/home/rithomas/cache/ABC/6_8_Meter/temp_abc.abc' -c

    if (len(formatted_tune) < 100):
        valid_tune = False
    
    print(error_logs)
    warnings = 0
    for log in error_logs:
        if 'Warning' in log:
            warnings = warnings + 1
        if 'Error' in log:
            valid_tune = False
            break

    if (warnings > MAX_WARNINGS):
        valid_tune = False

    if valid_tune:
        print(formatted_tune)
        abc_file.write(formatted_tune + '\n\n')
        i = i + 1

abc_file.close()
!rm /home/rithomas/cache/ABC/6_8_Meter/temp_abc.abc

/home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_512/lstm.json
{'tune_embedding_size': '16', 'rnn': DictWrapper({'num_layers': '3', 'num_units': '512', 'unit_type': 'lstm'})}
Restored from /home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_512/ckpt/ckpt-22
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:1
M:6/8
K:Cmajor
L:1/8
|:G|c2cB2G|AcAGFE|F2dE2c|D2cBAB|c2cB2G|AcAGFE|F2dE2c|BABc2:||:G|c2eg2a|gecg2e|G2Bd2e|dBGd2B|c2eg2a|gecg2e|fafege|dcBc2:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-10 : Track 0 Bar 1 has 5 units instead of 6', 'Warning in line-char 5-10 : Track 0 Bar 1 has 5 units instead of 6 in repeat', 'Error in line-char 6-107 : Missing :| at end of tune']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-8 : Track 0 Bar 0 has 11 units instead of 6 in repeat']
X:

['4.02 February 22 2017 abc2midi', 'Warning in line-char 6-111 : Assuming repeat']
X:14
M:6/8
K:Gmajor
L:1/8
|:B3BGE|cGcAFD|B3BAG|dAGAFD|B3BGE|c3ABc|dedcAF|AGFG3:|||d2ABcA|dgdBAG|dBcdBA|BAGABc|BcdABc|dedcBA|BABcAF|AGFG2A:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Error in line-char 6-239 :  found [1 or like before |:', 'Warning in line-char 6-239 : Assuming repeat', 'Warning in line-char 6-239 : replacing |: with double repeat (::)', 'Warning in line-char 5-48 : Track 0 Bar 5 has 11 units instead of 6', 'Warning in line-char 5-48 : Track 0 Bar 5 has 11 units instead of 6 in repeat', 'Error in line-char 5-66 : Expected end repeat not found at |:']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:15
M:6/8
K:Gmajor
L:1/8
DGGGFG|Addd2e|fefded|cAGAGF|D2GGFG|Addd2g|fedcAF|AGFGFE|DGGGFG|ABcd2e|fafded|cAGFGA|BABBAG|ABcd2f|fedcAF|AGFGBd||gdggdg|gdgbag|fefded|cAGF2g|gdggdg|gdgbag|fedcAF|AGFGBd|gdggdg|gdga2

['4.02 February 22 2017 abc2midi']
X:28
M:6/8
K:Gmajor
L:1/8
|:BdddBA|BdddBA|Beeefg|fede2f|gbgedB|BABGAc|BABGED|EGGG2A:||:BABGED|EGGG2A|B3GED|EAAABc|BABGED|EGGGAB|dBedBA|BGGG3:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:29
M:6/8
K:Gmajor
L:1/8
|:d3def|edBdBA|d3edB|dBAABc|d3def|edBdef|g2efdB|1A3ABc:|2A3A2f|||:g2efed|e3edB|g2efdB|A3ABd|g2efed|ec/A/eefg|ageedB|dBAA3:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-29 : Track 0 Bar 4 has 7 units instead of 6', 'Warning in line-char 5-29 : Track 0 Bar 4 has 7 units instead of 6 in repeat']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:30
M:6/8
K:Dmajor
L:1/8
|:e|f3dBA|BdBAFA|fefdBA|Bede3|defdBA|BdBAFA|B3dBA|Fdcd2:||:E|F3Bdf|bfefdB|AFABef|gededB|FGABdf|bfefdB|f3dBA|Bdcd2:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:31
M:6/8
K:Dmajor
L:1/

['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-2 : Track 0 Bar 0 has 2 units instead of 6 in repeat']
X:46
M:6/8
K:Bminor
L:1/8
Fe|cBcdcB|Afff2e|fBBcBA|BcBB2A|ABcdce|dffecB|ABAFBc|Bfed3:||:BABcBB|BcBfBB|cBAcAA|ecAecA|BBBdcB|BBff2B|cBcdcB|ABcB3:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-1 : Track 0 Bar 0 has 1 units instead of 6 in repeat']
X:47
M:6/8
K:Cmajor
L:1/8
F|E/F/GCDEF|GcABcd|efdga/g/e|fdcBGF|E2CEFD|GCDEFG|Bc/d/BcBG|FDB,C3:||:EGFG3|AdBcBA|G3GFG|AdBcAG|EAEFGA|BcdBGG|ABcdcB|AG^FG3:||
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-145 : Track 0 Bar 19 has 7 units instead of 6 in repeat']
X:48
M:6/8
K:Gmajor
L:1/8
DEGAGA|BAGABe|d2BBAG|A2BAGE|DEGAGA|B(3GFGB2g|dBGAGE|DEGG3:||:g2gedB|edeg2e|dBGdBG|cdBABd|g2ge2d|Bged2e|dBGAGA|BGFG3:|g2ee2g|edBd2e|gedBAG|AGABGED|G2DE2D|G2dd2e|dBAAGA|GGGG3||
-------------------------------------------------

['4.02 February 22 2017 abc2midi', 'Error in line-char 6-309 :  found [1 or like before |:', 'Warning in line-char 6-309 : Assuming repeat', 'Warning in line-char 6-309 : replacing |: with double repeat (::)', 'Warning in line-char 5-62 : Track 0 Bar 8 has 15/2 units instead of 6', 'Warning in line-char 5-62 : Track 0 Bar 8 has 15/2 units instead of 6 in repeat', 'Warning in line-char 5-193 : Track 0 Bar 25 has 10 units instead of 6', 'Warning in line-char 5-186 : Track 0 Bar 24 has 9 units instead of 6 in repeat', 'Warning in line-char 5-201 : Track 0 Bar 25 has 11 units instead of 6 in repeat', 'Warning in line-char 5-193 : Track 0 Bar 25 has 8 units instead of 6', 'Warning in line-char 5-252 : Track 0 Bar 33 has 4 units instead of 6', 'Warning in line-char 5-252 : Track 0 Bar 33 has 4 units instead of 6 in repeat']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:61
M:6/8
K:Gmajor
L:1/8
d|BGBdBG|cGgdBG|DGBdBG|cAFdAF|G2BdBg|ecgdBG|BGBdBA|GFAG

['4.02 February 22 2017 abc2midi']
X:70
M:6/8
K:Edorian
L:1/8
|:F|GEFG2A|B2edBd|edBGAB|GBAGED|E^DEG2A|BABdef|edBAGA|BGFG2:||:e|fgaefg|fagfed|Bdef2f|gfef2g|eaagfe|fagfed|edBdBA|BGFG2:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-61 : Track 0 Bar 8 has 5 units instead of 6', 'Warning in line-char 5-61 : Track 0 Bar 8 has 5 units instead of 6 in repeat']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:71
M:6/8
K:Amajor
L:1/8
AFEA3|Bcec2a|BcecBA|BAFAFE|AAEFAB|cAeecA|BcecBA|1BAFE2F:|2AFEABc|]|:e2aecB|AacBBc|e2ec2A|cdeecA|a2af2e|aeccBA|B2ec2B|1BAGAce:|2BAGFAE||
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 6-116 : Assuming repeat', 'Warning in line-char 6-116 : replacing |: with double repeat (::)', 'Warning in line-char 5-63 : Track 0 Bar 8 has 7 units instead of 6', 'Warning in line-char 5-71 : Track 0 Bar 9 has 7 u

['4.02 February 22 2017 abc2midi', 'Error in line-char 5-44 : Unrecognized character: 2', 'Error in line-char 5-174 : Cannot apply broken rhythm', 'Warning in line-char 5-50 : Track 0 Bar 5 has 10 units instead of 6', 'Warning in line-char 5-5 : Track 0 Bar 0 has 8 units instead of 6 in repeat', 'Warning in line-char 5-50 : Track 0 Bar 5 has 10 units instead of 6 in repeat', 'Warning in line-char 5-125 : Track 0 Bar 14 has 7 units instead of 6', 'Warning in line-char 5-125 : Track 0 Bar 14 has 7 units instead of 6 in repeat', 'Warning in line-char 5-147 : Track 0 Bar 16 has 7 units instead of 6', 'Warning in line-char 5-171 : Track 0 Bar 19 has 9/2 units instead of 6', 'Warning in line-char 5-147 : Track 0 Bar 16 has 7 units instead of 6 in repeat', 'Warning in line-char 5-171 : Track 0 Bar 19 has 9/2 units instead of 6 in repeat']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 5-118 : Track 0 Bar 16 has 5 units instead of

['4.02 February 22 2017 abc2midi']
X:92
M:6/8
K:Gmajor
L:1/8
|:B2BBcd|GAGE2D|GBdGBd|ABAA2d|B2BBcd|GAGE2D|GBdedB|ABAG3:||:e2eefg|defgfe|edeg2e|dBGBBd|e2defg|dagfgf|e2dedB|ABAA2d:|
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Warning in line-char 3-0 : Ignoring text: A', 'Warning in line-char 5-0 : Ignoring text: da|:A2AA2G|E2FG3|A2AA2B|c2de2d|c2AB2G|E2FG3|GABcBA|G3E3:||:e2fgfe|d2cB2A|BcBA2A|GAGF3|g2fgfe|e2cB2B|c2AB3|G3A3:|', 'Error in line-char 6-0 : No valid K: field found at start of tune']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi', 'Error in line-char 6-128 :  found another |: after a |:', 'Warning in line-char 6-128 : Assuming repeat', 'Warning in line-char 6-128 : replacing with double repeat (::)', 'Error in line-char 6-128 : Missing :| at end of tune']
-----------------------------------------------------
['4.02 February 22 2017 abc2midi']
X:93
M:6/8
K:Gdorian
L:1/8
|:B3BAF|GFDCDF|GDADBD|cB